In [37]:
import io
import nbformat
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier

from tqdm import tqdm
import re
import csv

In [23]:
data = "train_data.csv"
df1 = pd.read_csv(data)

In [29]:
data = "train_data.csv"
df1 = pd.read_csv(data)

#Prepare card id and ip id for plotting

# # 1.Card ID
# card_enc = LabelBinarizer()
# df1['card_id'] = card_enc.fit_transform(df1[])

#
# # 2.IP ID
# ip_enc = LabelBinarizer()
# df1['ip_id'] = ip_enc.fit_transform(df1.ip_id)

#Encoding, Cleaning the data
df1.loc[df1['cardverificationcodesupplied'].isna(),'cardverificationcodesupplied'] = False
df1.loc[df1['issuercountrycode'].isna(),'issuercountrycode'] = 'ZZ'
df1.loc[df1['shoppercountrycode'].isna(),'shoppercountrycode'] = 'ZZ'

#Feature Engineering
df1['countries_equal'] = (df1['shoppercountrycode'] == df1['issuercountrycode'])
df1.loc[df1['countries_equal'] == False,'countries_equal'] = 0
df1.loc[df1['countries_equal'] == True,'countries_equal'] = 1

unique_issuer_cc = df1['issuercountrycode'].unique()
unique_shopper_cc = df1['shoppercountrycode'].unique()
both = np.append(unique_issuer_cc, unique_shopper_cc)
df_countrycodes = pd.DataFrame(both)
unique_codes = df_countrycodes[0].unique()
lb = LabelBinarizer()
y = lb.fit_transform(df1['issuercountrycode'])

# Create a new DataFrame with the transformed 'issuercountrycode' column
df2 = pd.DataFrame(y, columns=['issuercountrycode_'+str(i) for i in range(y.shape[1])])

# Replace the 'issuercountrycode' column in the original DataFrame with the transformed column
df1 = pd.concat([df1.drop('issuercountrycode', axis=1), df2], axis=1)

y = lb.fit_transform(df1['shoppercountrycode'])

# Create a new DataFrame with the transformed 'issuercountrycode' column
df2 = pd.DataFrame(y, columns=['shoppercountrycode_'+str(i) for i in range(y.shape[1])])

# Replace the 'issuercountrycode' column in the original DataFrame with the transformed column
df1 = pd.concat([df1.drop('shoppercountrycode', axis=1), df2], axis=1)

def conv(row):
    currency_dict = {"BGN": 1.9558, "NZD": 1.6805, "ILS": 4.0448, "RUB": 72.2099, "CAD": 1.5075, "USD": 1.1218,
                     "PHP": 58.125, "CHF": 1.1437, "ZAR": 16.0224, "AUD": 1.5911, "JPY": 124.93, "TRY": 6.6913,
                     "HKD": 8.8007, "MYR": 4.6314, "THB": 35.802, "HRK": 7.413, "NOK": 9.6678, "IDR": 15953.68,
                     "DKK": 7.4646, "CZK": 25.659, "HUF": 322.97, "GBP": 0.86248, "MXN": 21.2829, "KRW": 1308.01,
                     "ISK": 136.2, "SGD": 1.5263, "BRL": 4.405, "PLN": 4.2868, "INR": 78.0615, "RON": 4.7596,
                     "CNY": 7.5541, "SEK": 10.635}
    return row['amount'] / (currency_dict[row['currencycode']]*100)

df1['amount_eur'] = df1.apply(lambda x: conv(x), axis=1)

lb2 = LabelBinarizer()
y = lb2.fit_transform(df1['txvariantcode'])

# Create a new DataFrame with the transformed 'txvariantcode' column
df2 = pd.DataFrame(y, columns=['txvariantcode_'+str(i) for i in range(y.shape[1])])

# Replace the 'txvariantcode' column in the original DataFrame with the transformed column
df1 = pd.concat([df1.drop('txvariantcode', axis=1), df2], axis=1)

lb3 = LabelBinarizer()
y = lb3.fit_transform(df1['currencycode'])

# Create a new DataFrame with the transformed 'currencycode' column
df2 = pd.DataFrame(y, columns=['currencycode_'+str(i) for i in range(y.shape[1])])

# Replace the 'currencycode' column in the original DataFrame with the transformed column
df1 = pd.concat([df1.drop('currencycode', axis=1), df2], axis=1)

lb4 = LabelBinarizer()
y = lb4.fit_transform(df1['shopperinteraction'])

# Create a new DataFrame with the transformed 'shopperinteraction' column
df2 = pd.DataFrame(y, columns=['shopperinteraction_'+str(i) for i in range(y.shape[1])])

# Replace the 'shopperinteraction' column in the original DataFrame with the transformed column
df1 = pd.concat([df1.drop('shopperinteraction', axis=1), df2], axis=1)

df1['accountcode'] = df1['accountcode'].apply(lambda x: re.sub('Account','',x))
df1['accountcode_cc'] = 0
df1.loc[(df1['accountcode'] == 'UK'),'accountcode_cc'] = 'GB'
df1.loc[(df1['accountcode'] == 'Mexico'),'accountcode_cc'] = 'MX'
df1.loc[(df1['accountcode'] == 'Sweden'),'accountcode_cc'] = 'SE'
df1.loc[(df1['accountcode'] == 'APAC'),'accountcode_cc'] = 'APAC'

lb5 = LabelBinarizer()
y = lb5.fit_transform(df1['accountcode_cc'])

# Create a new DataFrame with the transformed 'accountcode_cc' column
df2 = pd.DataFrame(y, columns=['accountcode_cc_'+str(i) for i in range(y.shape[1])])

# Replace the 'accountcode_cc' column in the original DataFrame with the transformed column
df1 = pd.concat([df1.drop('accountcode_cc', axis=1), df2], axis=1)

df1.loc[df1['mail_id'].str.contains('na',case=False),'mail_id'] = 'email99999'

df1.loc[df1['cvcresponsecode'] > 2,'cvcresponsecode'] = 3


# drop id, mail_id, ip_id, card_id
df1 = df1.drop(['Id','mail_id','ip_id','card_id', 'amount', 'accountcode'], axis=1)

In [30]:
# classify majority and minority class
minority_class = df1[df1['label'] == 1]
majority_class = df1[df1['label'] == 0]

In [31]:
# calculate the difference between the two classes
nsamples = len(majority_class) - len(minority_class)

# get samples from the minority class
samples = minority_class.sample(n=nsamples, replace=True)

print(nsamples)
print(samples)

189076
             bin cardverificationcodesupplied  cvcresponsecode  label  \
85236   557351.0                         True                1      1   
94926   547046.0                         True                0      1   
131387  414726.0                         True                0      1   
7245    475130.0                        False                0      1   
34761   516310.0                         True                0      1   
...          ...                          ...              ...    ...   
91835   476367.0                        False                0      1   
112623  476367.0                        False                0      1   
186984  492910.0                         True                1      1   
23065   455704.0                         True                0      1   
166939  477291.0                         True                0      1   

       countries_equal  issuercountrycode_0  issuercountrycode_1  \
85236                1                    0     

In [32]:
# find k nearest neighbors of each sample in the minority class
neigh = NearestNeighbors(n_neighbors=5)
nbrs = neigh.fit(minority_class)
distances, indices = nbrs.kneighbors(samples)

# create synthetic samples
synthetic_samples = []
for i in tqdm(range(len(samples))):
    nn = indices[i]
    nn_index = np.random.choice(nn)
    diff = minority_class.iloc[nn_index] - samples.iloc[i]
    r = np.random.rand()
    synthetic_sample = samples.iloc[i] + r * diff
    synthetic_samples.append(synthetic_sample)
    
# convert the synthetic samples to a dataframe
synthetic_samples = pd.DataFrame(synthetic_samples, columns=df1.columns)

# add the synthetic samples to the minority class
oversampled = pd.concat([minority_class, synthetic_samples])

100%|██████████| 189076/189076 [01:43<00:00, 1819.54it/s]


In [33]:
oversampled.head()

# print(oversampled.shape)

,bin,cardverificationcodesupplied,cvcresponsecode,label,countries_equal,issuercountrycode_0,issuercountrycode_1,issuercountrycode_2,issuercountrycode_3,issuercountrycode_4,...,currencycode_2,currencycode_3,currencycode_4,shopperinteraction_0,shopperinteraction_1,shopperinteraction_2,accountcode_cc_0,accountcode_cc_1,accountcode_cc_2,accountcode_cc_3
827,552033.0,True,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1132,537196.0,True,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1632,532655.0,True,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3219,520416.0,True,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3370,483741.0,True,0.0,1.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [56]:
# combine with majority class
smoted = pd.concat([majority_class, oversampled])

print(smoted.shape)

# smoted.head()

smoted = smoted.sort_values(smoted.columns[0])
# Get the number of rows in the DataFrame
n_rows = len(smoted)

# Split the DataFrame in half
half1 = smoted.iloc[:n_rows // 1]

print(len(half1))
smoted = half1

(378704, 280)
378704


In [57]:
# split the smoted data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(smoted, smoted['label'], test_size=0.2, random_state=42)

# # scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # train a logistic regression model on the oversampled data
# clf = LogisticRegression()
# clf.fit(X_train, y_train)


In [58]:
# use a neural network for classification
# Create and train the Multi-Layer Perceptron Classifier
clf = MLPClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Print the classification report and accuracy
print(classification_report(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     37882
         1.0       0.99      0.99      0.99     37859

    accuracy                           0.99     75741
   macro avg       0.99      0.99      0.99     75741
weighted avg       0.99      0.99      0.99     75741

Accuracy: 0.9866518794312196
